<a href="https://colab.research.google.com/github/sreetabasu1/Regime-Aware-Feature-Selection-Carry-Trade/blob/main/VAEQuantBrokers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#code formatting

import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from scipy.stats import norm

# Load data from the Excel file into a DataFrame with headers
file_path = 'Currency12_07.csv'
exchange_df = pd.read_csv(file_path, header=0, index_col=0, parse_dates=True)

# Calculate returns for each column
returns_df = exchange_df.diff() / exchange_df.shift(1)
returns_df = returns_df.iloc[:, :]
returns_df.fillna(0, inplace=True)


cutoff = pd.to_datetime('2018-12-30', format='%Y-%m-%d')
X_train = returns_df[returns_df.index < cutoff]
X_test = returns_df[returns_df.index >= cutoff]



Prompt: Using Tensorflow for GM- VAE
Errors in the below code and how they were fixed:
Error 1: Issues with defining X_train
Input Size Initialization:

input_size = X_train.size should be replaced with input_size = X_train.shape[1] to get the number of features.
Data Loading in DataLoader:

The DataLoader should work with PyTorch Datasets, but in your case, X_train seems to be a DataFrame. You can convert it to a PyTorch Tensor using torch.from_numpy(X_train.values).

convert X_train to Tensor: data = data.float().view(-1, input_size)

Error 2: target must be between 0 and 1
Fix: switch reconstruction loss to MSE

This is running regular Gaussian prior, but we standardize data and recon_batch (runtime inputs of the loss function) so that we can use cross entropy error instead of MSE.

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Assuming X_train is a DataFrame
# Replace this line with your actual data loading

# Standardize the input data
scaler = StandardScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train.values), columns=X_train.columns)

# Define VAE model
class VAE(nn.Module):
    def __init__(self, input_size, hidden_size, latent_size):
        super(VAE, self).__init__()

        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, latent_size * 2)  # 2 for mean and log-variance
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(latent_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, input_size),
            nn.Sigmoid()  # Output between 0 and 1 for image data
        )

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        # Encode
        encoded = self.encoder(x)

        # Split into mean and log-variance
        mu, logvar = torch.chunk(encoded, 2, dim=1)

        # Reparameterize
        z = self.reparameterize(mu, logvar)

        # Decode
        decoded = self.decoder(z)

        return decoded, mu, logvar

# Hyperparameters
input_size = X_train_scaled.shape[1]
hidden_size = 256
latent_size = 2  # Dimensionality of the latent space

# Build VAE model
vae = VAE(input_size, hidden_size, latent_size)

# Loss function
def loss_function(recon_x, x, mu, logvar):
    BCE = nn.BCELoss(reduction='sum')(recon_x, x)  # BCELoss for probabilities
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

# Optimizer
optimizer = optim.Adam(vae.parameters(), lr=0.001)

# Convert X_train_scaled to PyTorch Tensor
X_train_tensor = torch.from_numpy(X_train_scaled.values).float()

# Create DataLoader
train_dataset = TensorDataset(X_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    total_loss = 0
    for batch_idx, (data,) in enumerate(train_loader):  # Remove the unnecessary label _
        data = torch.sigmoid(data)  # Apply sigmoid to input data
        optimizer.zero_grad()
        recon_batch, mu, logvar = vae(data)
        recon_batch = torch.sigmoid(recon_batch)  # Apply sigmoid to reconstructed batch
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        total_loss += loss.item()
        optimizer.step()

    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(train_loader.dataset)}')

# Sample from the VAE
with torch.no_grad():
    # Sample from the latent space
    z_sample = torch.randn(64, latent_size)
    # Decode the samples
    sample = torch.sigmoid(vae.decoder(z_sample)).view(64, 1, input_size)  # Apply sigmoid to generated samples
    # Save or visualize the generated samples as needed


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch 1/10, Loss: 11.835940591479007
Epoch 2/10, Loss: 11.78659973667989
Epoch 3/10, Loss: 11.784750558042752
Epoch 4/10, Loss: 11.784175801597105
Epoch 5/10, Loss: 11.783960112432288
Epoch 6/10, Loss: 11.783952045026911
Epoch 7/10, Loss: 11.783747212988567
Epoch 8/10, Loss: 11.78368230424607
Epoch 9/10, Loss: 11.783811419682046
Epoch 10/10, Loss: 11.78356199521444


This is the code for **Gaussian mixture** prior, using the loss function $L(\lambda) =  \Sigma_y q(y|x) (\log(y|x) - \log p(y) + \log q(z|x, y) - \log p(z|y) - \log p(x|y, z))$

from Quantitative Brokers

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Assuming X_train is a DataFrame
# Replace this line with your actual data loading

# Standardize the input data
scaler = StandardScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train.values), columns=X_train.columns)

# Define VAE model
class VAE(nn.Module):
    def __init__(self, input_size, hidden_size, latent_size):
        super(VAE, self).__init__()

        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, latent_size * 2)  # 2 for mean and log-variance
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(latent_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, input_size),
            nn.Sigmoid()  # Output between 0 and 1 for image data
        )

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        # Encode
        encoded = self.encoder(x)

        # Split into mean and log-variance
        mu, logvar = torch.chunk(encoded, 2, dim=1)

        # Reparameterize
        z = self.reparameterize(mu, logvar)

        # Decode
        decoded = self.decoder(z)

        return decoded, mu, logvar

# Hyperparameters
input_size = X_train_scaled.shape[1]
hidden_size = 256
latent_size = 2  # Dimensionality of the latent space

# Build VAE model
vae = VAE(input_size, hidden_size, latent_size)

# Loss function
def loss_function(recon_x, x, mu, logvar):
    # Reconstruction loss term
    BCE = nn.BCELoss(reduction='sum')(recon_x, x)

    # Latent space regularization terms
    KLD_y = -0.5 * torch.sum(1 + logvar[:, :latent_size] - mu[:, :latent_size].pow(2) - logvar[:, :latent_size].exp())
    KLD_z = -0.5 * torch.sum(1 + logvar[:, latent_size:] - mu[:, latent_size:].pow(2) - logvar[:, latent_size:].exp())

    # Overall loss
    loss = BCE + KLD_y + KLD_z
    return loss

# Optimizer
optimizer = optim.Adam(vae.parameters(), lr=0.001)

# Convert X_train_scaled to PyTorch Tensor
X_train_tensor = torch.from_numpy(X_train_scaled.values).float()

# Create DataLoader
train_dataset = TensorDataset(X_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    total_loss = 0
    for batch_idx, (data,) in enumerate(train_loader):  # Remove the unnecessary label _
        data = torch.sigmoid(data)  # Apply sigmoid to input data
        optimizer.zero_grad()
        recon_batch, mu, logvar = vae(data)
        recon_batch = torch.sigmoid(recon_batch)  # Apply sigmoid to reconstructed batch
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        total_loss += loss.item()
        optimizer.step()

    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(train_loader.dataset)}')

# Sample from the VAE
with torch.no_grad():
    # Sample from the latent space
    z_sample = torch.randn(64, latent_size)
    # Decode the samples
    sample = torch.sigmoid(vae.decoder(z_sample)).view(64, 1, input_size)  # Apply sigmoid to generated samples
    # Save or visualize the generated samples as needed


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch 1/10, Loss: 11.835940591479007
Epoch 2/10, Loss: 11.78659973667989
Epoch 3/10, Loss: 11.784750558042752
Epoch 4/10, Loss: 11.784175801597105
Epoch 5/10, Loss: 11.783960112432288
Epoch 6/10, Loss: 11.783952045026911
Epoch 7/10, Loss: 11.783747212988567
Epoch 8/10, Loss: 11.78368230424607
Epoch 9/10, Loss: 11.783811419682046
Epoch 10/10, Loss: 11.78356199521444


This is the code for $\beta$-VAE with loss function given by \begin{align*}
\mathcal{L}(\lambda) &= \sum_y q(y|x) \left( \beta \cdot (\log q(y|x) - \log p(y) + \log q(z|x, y) - \log p(z|y)) - \log p(x|y, z) \right) \\
\end{align*}.

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Assuming X_train is a DataFrame
# Replace this line with your actual data loading

# Standardize the input data
scaler = StandardScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train.values), columns=X_train.columns)

# Define VAE model
class VAE(nn.Module):
    def __init__(self, input_size, hidden_size, latent_size, beta):
        super(VAE, self).__init__()

        self.beta = beta

        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, latent_size * 2)  # 2 for mean and log-variance
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(latent_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, input_size),
            nn.Sigmoid()  # Output between 0 and 1 for image data
        )

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        # Encode
        encoded = self.encoder(x)

        # Split into mean and log-variance
        mu, logvar = torch.chunk(encoded, 2, dim=1)

        # Reparameterize
        z = self.reparameterize(mu, logvar)

        # Decode
        decoded = self.decoder(z)

        return decoded, mu, logvar

# Hyperparameters
input_size = X_train_scaled.shape[1]
hidden_size = 256
latent_size = 2  # Dimensionality of the latent space
beta = 1.0  # Adjust based on your problem and data characteristics

# Build VAE model
vae = VAE(input_size, hidden_size, latent_size, beta)

# Loss function
def loss_function(recon_x, x, mu, logvar, beta):
    BCE = nn.BCELoss(reduction='sum')(recon_x, x)
    KLD_y = -0.5 * torch.sum(1 + logvar[:, :latent_size] - mu[:, :latent_size].pow(2) - logvar[:, :latent_size].exp())
    KLD_z = -0.5 * torch.sum(1 + logvar[:, latent_size:] - mu[:, latent_size:].pow(2) - logvar[:, latent_size:].exp())
    return BCE + beta * (KLD_y + KLD_z)

# Optimizer
optimizer = optim.Adam(vae.parameters(), lr=0.001)

# Convert X_train_scaled to PyTorch Tensor
X_train_tensor = torch.from_numpy(X_train_scaled.values).float()

# Create DataLoader
train_dataset = TensorDataset(X_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)

# Training loop
num_epochs = 10
beta = 3
for epoch in range(num_epochs):
    total_loss = 0
    for batch_idx, (data,) in enumerate(train_loader):
        data = torch.sigmoid(data)  # Apply sigmoid to input data
        optimizer.zero_grad()
        recon_batch, mu, logvar = vae(data)
        recon_batch = torch.sigmoid(recon_batch)  # Apply sigmoid to reconstructed batch
        loss = loss_function(recon_batch, data, mu, logvar, beta)
        loss.backward()
        total_loss += loss.item()
        optimizer.step()

    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(train_loader.dataset)}')

# Sample from the VAE
with torch.no_grad():
    # Sample from the latent space
    z_sample = torch.randn(64, latent_size)
    # Decode the samples
    sample = torch.sigmoid(vae.decoder(z_sample)).view(64, 1, input_size)  # Apply sigmoid to generated samples
    # Save or visualize the generated samples as needed


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch 1/10, Loss: 11.855083158814622
Epoch 2/10, Loss: 11.786603505574615
Epoch 3/10, Loss: 11.78506128247457
Epoch 4/10, Loss: 11.784319426046107
Epoch 5/10, Loss: 11.78415054630745
Epoch 6/10, Loss: 11.783908037638499
Epoch 7/10, Loss: 11.783639423831081
Epoch 8/10, Loss: 11.783818470963844
Epoch 9/10, Loss: 11.783644756940284
Epoch 10/10, Loss: 11.783697607682988
